## 控制迷宫寻宝机器人

项目内容：编写代码，控制一个机器人，在模拟环境中行走，并找到目标宝藏。

机器人所在的模拟环境中，会包含几个因素：机器人的起点、障碍物、宝藏箱。任务可拆分为：

1. 分析模拟环境的数据
2. 控制机器人随机行动

---

---

## 第一节 分析模拟环境的数据

首先，只有足够了解机器人所在的环境，机器人才能成功找到目标宝藏，所以先对机器人所在环境的数据进行分析。这个部分，主要应用数据结构和控制流。



### 1.1 理解模拟环境数据的储存格式

将模拟环境抽象成一个格子世界，每个格子按照坐标编号进行标记；每个格子中会有四个情况，分别为普通格子（可通行）、机器人的起点（可通行）、障碍物（不可通行）、宝藏箱（目标点）。例如，一个模拟环境就可以抽象成3行4列的格子世界，并按这按这样的方法进行存储：
```
environment = [[0,0,0,2], 
               [1,2,0,0],
               [0,2,3,2]]
```
用一个列表来保存虚拟世界的数据。外层列表中的每一个元素依然是一个列表，它代表模拟环境中每行的数据。而对于这个列表中的每个元素都是一个数，它们的含义是：
- 0: 普通格子（可通行）
- 1: 机器人的起点（可通行）
- 2: 障碍物（不可通行）
- 3: 宝藏箱（目标点）

那么，根据上述的数据，这个迷宫的第二行第一列，是机器人的起点。

__注：描述的迷宫的坐标位置（第一行第一列），和迷宫下标索引的值（如 `(0,0)`）是不一样的，注意下标的问题。__


如下的代码，使用了辅助函数，读取了模拟环境的数据，并保存在了 `env_data` 变量中。


In [1]:
import helper

env_data = helper.fetch_maze()

maze-id 1-1529301262
[[3, 2, 2, 2, 2, 2, 2, 2, 1],
 [0, 0, 2, 2, 2, 2, 2, 0, 0],
 [2, 0, 0, 2, 2, 2, 0, 0, 2],
 [2, 2, 0, 0, 2, 0, 0, 2, 2],
 [2, 2, 2, 0, 0, 0, 2, 2, 2]]


---


**task 1：**在如下代码中，写代码获得这些值：

1. 模拟环境的长和宽
2. 模拟环境中第3行第6列元素

In [2]:
#TODO 1模拟环境的行数
rows = len(env_data)

#TODO 2模拟环境的列数
columns = len(env_data[0])

#TODO 3取出模拟环境第三行第六列的元素
row_3_col_6 = env_data[2][5] 

print("迷宫共有", rows, "行", columns, "列，第三行第六列的元素是", row_3_col_6)

迷宫共有 5 行 9 列，第三行第六列的元素是 2


---

## 1.2 分析模拟环境数据

接着需要对模拟环境的中的数据进行分析。

---

**task 2：**在如下代码中，计算模拟环境中，第一行和第三列的障碍物个数。

In [3]:
#TODO 4计算模拟环境中，第一行的的障碍物个数。
number_of_barriers_row1 = len([i for i in env_data[0] if i == 2])
# 更简练的方法：直接计算 2 的个数
number_of_barriers_row1 = env_data[0].count(2)

#TODO 5计算模拟环境中，第三列的的障碍物个数。
number_of_barriers_col3 = len([i[2] for i in env_data if i[2] == 2])
# 更简练的方法: 先转置，再计算第三行的 2 的个数
number_of_barriers_col3 = list(zip(*env_data))[2].count(2)

print("迷宫中，第一行共有", number_of_barriers_row1, "个障碍物，第三列共有", number_of_barriers_col3, "个障碍物。")

迷宫中，第一行共有 7 个障碍物，第三列共有 3 个障碍物。


---

**task 3：**在如下代码中：

1. 创建一个名为 `loc_map` 的字典，它有两个键值，分别为 `start` 和 `destination`，对应的值分别为起点和目标点的坐标，它们以如 `(1,1)` 的形式保存为元组。
2. 从字典中取出 `start` 对应的值，保存在 `robot_current_loc` 对应的变量中，这个变量表示小车现在的位置。

In [4]:
# 查找 起始点 和 目标点 的坐标
for i, row in enumerate(env_data):
    for j, ele in enumerate(row):
        if ele == 1:
            start = (i+1, j+1)
        if ele == 3:
            destination = (i+1, j+1)
            
loc_map = {'start':start, 'destination':destination} #TODO 6按照上述要求创建字典

robot_current_loc = loc_map['start'] #TODO 7保存机器人当前的位置

In [5]:
robot_current_loc

(1, 9)

In [6]:
loc_map['destination']

(1, 1)


---

---

## 第二节 控制机器人随机漫步

在这一步中，需发出指令，控制机器人在环境中随机行动。这个部分主要应用了控制流、调用函数的知识。



## 2.1 控制机器人行动

机器人能够执行四个动作：向上走 `u`、向下走 `d`、向左走 `l`、向右走 `r`。但是，由于有障碍，很多时候机器人的行动并不能成功。所以在这里，需要实现一个函数，来判断机器人在某个位置，执行某个移动动作是否可行。

---

**task 4：**在下方代码中，实现名为 `is_move_valid_special` 的函数，它有两个输入，分别为机器人所在的位置坐标 `loc`，以及即将执行的动作 `act`，如 `(1,1)` 及 `u`。接着它的返回是一个布尔值，表明小车在 `loc` 位置下，是否可以执行动作 `act`。


tip 1：*可以读取上方定义的 `env_data` 变量，来读取模拟环境的数据。*

tip 2：*需要处理边界的情况，即机器人走到了虚拟环境边界时，是不能够走出虚拟环境的。*

In [7]:
def is_move_valid_sepcial(loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 8
    
    # “向上走”
    if act == 'u':
        # 判断边界，如果位于第一行，返回False
        if loc[0] == 1: return False
        else:
            # 向上一格的数字
            upper_num = env_data[loc[0]-2][loc[1]-1]
            if upper_num == 2: return False
            else: return True
            
    # “向下走”       
    if act == 'd':
        # 判断边界，如果位于最后一行，返回False
        if loc[0] == len(env_data): return False
        else:
            # 向下一格的数字
            down_num = env_data[loc[0]][loc[1]-1]
            if down_num == 2: return False
            else: return True
    
    # “向左走”       
    if act == 'l':
        # 判断边界，如果位于第一列，返回False
        if loc[1] == 1: return False
        else:
            # 向左一格的数字
            left_num = env_data[loc[0]-1][loc[1]-2]
            if down_num == 2: return False
            else: return True
    
    # “向右走”       
    if act == 'r':
        # 判断边界，如果位于最后一列，返回False
        if loc[1] == len(env_data[0]): return False
        else:
            # 向右一格的数字
            right_num = env_data[loc[0]-1][loc[1]]
            if right_num == 2: return False
            else: return True
    

---

**task 5：**在下方代码中，重新实现一个名为 `is_move_valid` 的函数，它有三个输入，分别为模拟环境的数据 `env_data`、机器人所在的位置坐标 `loc`、以及即将执行的动作 `act`。它的返回值与此前一样，是一个布尔值，表明小车在给定的虚拟环境中的 `loc` 位置下，是否可以执行动作 `act`。

In [8]:
def is_move_valid(env_data, loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    env -- list, the environment data
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 9
    # is_move_valid_sepcial()的实现方法 太愚蠢，这个更紧凑、简洁、易懂
    
    x, y = loc
    if act == 'u':
        x -= 1
    elif act == 'd':
        x += 1
    elif act == 'l':
        y -= 1
    elif act == 'r':
        y += 1

    return (1 <= x <= rows) and (1 <= y <= columns) and (env_data[x-1][y-1] != 2)

---

**task 6：****在任务5及任务6中的实现的两个函数中，`env_data` 这个变量有什么不同？**

is_move_valid_sepcial()函数中使用的env_data是全局变量，如果函数内部对其做一些更改操作，会直接影响到env_data；is_move_valid()函数中使用的env_data是局部变量，全局变量的env_data只能将数值传递到is_move_valid()函数内部，在函数内部对env_data进行增删操作也不会影响全局变量。

---

## 2.2 机器人可行动作

---

**task 7：**编写名为 `valid_actions` 的函数。它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`，输出是一个列表，表明机器人在这个位置所有的可行动作。

tip：*可调用上方定义的`is_move_valid`函数。*


In [9]:
## TODO 10 从头定义、实现你的函数
def valid_actions(env_data, loc):
    """
    Return all the valid actions for the robot
    at location loc.
    
    Keyword arguments:
    env_data -- list, the environment data
    loc -- tuple, robots current location
    """
    
    actions = ['u', 'd', 'l', 'r']
    return [act for act in actions if is_move_valid(env_data, loc, act)]

---

## 2.3 移动机器人

当机器人收到一个动作的时候，机器人的位置应发生相应的变化。

**task 8：**编写名为 `move_robot` 的函数，它有两个输入，分别为机器人当前所在的位置 `loc` 和即将执行的动作 `act`。接着会返回机器人执行动作之后的新位置 `new_loc`。

In [10]:
##TODO 11 从头定义、实现你的函数
def move_robot(loc, act):
    """
    Return the new location after the robot take
    action act.
    
    Keyword arguments:
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    if act == 'u':
        new_loc = (loc[0]-1, loc[1])
    if act == 'd':
        new_loc = (loc[0]+1, loc[1])
    if act == 'l':
        new_loc = (loc[0], loc[1]-1)
    if act == 'r':
        new_loc = (loc[0], loc[1]+1)
    
    return new_loc

---

## 2.4 随机移动机器人

接着，尝试在虚拟环境中随机移动机器人，看看会有什么效果。

**task 9：**编写名为 `random_choose_actions` 的函数，它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`。机器人会执行一个300次的循环，每次循环，他会执行以下任务：

1. 利用上方定义的 `valid_actions` 函数，找出当前位置下，机器人可行的动作；
2. 利用 `random` 库中的 `choice` 函数，从机器人可行的动作中，随机挑选出一个动作；
3. 接着根据这个动作，利用上方定义的 `move_robot` 函数，来移动机器人，并更新机器人的位置；
4. 当机器人走到终点时，输出“在第n个回合找到宝藏！”。

tip：如果机器人无法在300个回合内找到宝藏的话，增大这个数字，也许会有不错的效果 :P

In [11]:
import random

In [12]:
##TODO 12 从头实现你的函数
def random_choose_actions(env_data, loc):
    for i in range(1000):
        # 可选动作列表
        acts = valid_actions(env_data, loc)
        # 随机选择下一步动作
        act = random.choice(acts)
        # 更新位置
        loc = move_robot(loc, act)
        # 如果找到宝藏，则返回结果
        if loc == loc_map['destination']:
            return "在第{}个回合找到宝藏！".format(i)

In [13]:
# 运行
random_choose_actions(env_data, robot_current_loc)

'在第187个回合找到宝藏！'